# Unsupervised Lab Session

## Learning outcomes:
- Exploratory data analysis and data preparation for model building.
- PCA for dimensionality reduction.
- K-means and Agglomerative Clustering

## Problem Statement
Based on the given marketing campigan dataset, segment the similar customers into suitable clusters. Analyze the clusters and provide your insights to help the organization promote their business.

## Context:
- Customer Personality Analysis is a detailed analysis of a company’s ideal customers. It helps a business to better understand its customers and makes it easier for them to modify products according to the specific needs, behaviors and concerns of different types of customers.
- Customer personality analysis helps a business to modify its product based on its target customers from different types of customer segments. For example, instead of spending money to market a new product to every customer in the company’s database, a company can analyze which customer segment is most likely to buy the product and then market the product only on that particular segment.

## About dataset
- Source: https://www.kaggle.com/datasets/imakash3011/customer-personality-analysis?datasetId=1546318&sortBy=voteCount

### Attribute Information:
- ID: Customer's unique identifier
- Year_Birth: Customer's birth year
- Education: Customer's education level
- Marital_Status: Customer's marital status
- Income: Customer's yearly household income
- Kidhome: Number of children in customer's household
- Teenhome: Number of teenagers in customer's household
- Dt_Customer: Date of customer's enrollment with the company
- Recency: Number of days since customer's last purchase
- Complain: 1 if the customer complained in the last 2 years, 0 otherwise
- MntWines: Amount spent on wine in last 2 years
- MntFruits: Amount spent on fruits in last 2 years
- MntMeatProducts: Amount spent on meat in last 2 years
- MntFishProducts: Amount spent on fish in last 2 years
- MntSweetProducts: Amount spent on sweets in last 2 years
- MntGoldProds: Amount spent on gold in last 2 years
- NumDealsPurchases: Number of purchases made with a discount
- AcceptedCmp1: 1 if customer accepted the offer in the 1st campaign, 0 otherwise
- AcceptedCmp2: 1 if customer accepted the offer in the 2nd campaign, 0 otherwise
- AcceptedCmp3: 1 if customer accepted the offer in the 3rd campaign, 0 otherwise
- AcceptedCmp4: 1 if customer accepted the offer in the 4th campaign, 0 otherwise
- AcceptedCmp5: 1 if customer accepted the offer in the 5th campaign, 0 otherwise
- Response: 1 if customer accepted the offer in the last campaign, 0 otherwise
- NumWebPurchases: Number of purchases made through the company’s website
- NumCatalogPurchases: Number of purchases made using a catalogue
- NumStorePurchases: Number of purchases made directly in stores
- NumWebVisitsMonth: Number of visits to company’s website in the last month

### 1. Import required libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from scipy import stats
from sklearn.cluster import KMeans
from scipy.stats import zscore
from scipy.spatial import distance
from scipy.cluster.hierarchy import linkage,dendrogram,fcluster
from sklearn.cluster import AgglomerativeClustering
from sklearn.decomposition import PCA




### 2. Load the CSV file (i.e marketing.csv) and display the first 5 rows of the dataframe. Check the shape and info of the dataset.

In [2]:
import os
os.getcwd()

'C:\\Users\\Aashish'

In [3]:
df=pd.read_csv("marketing.csv")
df.head(3)

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response
0,5524,1957,Graduation,Single,58138.0,0,0,4/9/2012,58,635,...,10,4,7,0,0,0,0,0,0,1
1,2174,1954,Graduation,Single,46344.0,1,1,8/3/2014,38,11,...,1,2,5,0,0,0,0,0,0,0
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,...,2,10,4,0,0,0,0,0,0,0


In [4]:
os.chdir('C:\python\Lab 4 - Unsu[ervised learinng')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 27 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   2240 non-null   int64  
 1   Year_Birth           2240 non-null   int64  
 2   Education            2240 non-null   object 
 3   Marital_Status       2240 non-null   object 
 4   Income               2216 non-null   float64
 5   Kidhome              2240 non-null   int64  
 6   Teenhome             2240 non-null   int64  
 7   Dt_Customer          2240 non-null   object 
 8   Recency              2240 non-null   int64  
 9   MntWines             2240 non-null   int64  
 10  MntFruits            2240 non-null   int64  
 11  MntMeatProducts      2240 non-null   int64  
 12  MntFishProducts      2240 non-null   int64  
 13  MntSweetProducts     2240 non-null   int64  
 14  MntGoldProds         2240 non-null   int64  
 15  NumDealsPurchases    2240 non-null   i

### 3. Check the percentage of missing values? If there is presence of missing values, treat them accordingly.

In [6]:
df.isnull().sum()/len(df)*100

ID                     0.000000
Year_Birth             0.000000
Education              0.000000
Marital_Status         0.000000
Income                 1.071429
Kidhome                0.000000
Teenhome               0.000000
Dt_Customer            0.000000
Recency                0.000000
MntWines               0.000000
MntFruits              0.000000
MntMeatProducts        0.000000
MntFishProducts        0.000000
MntSweetProducts       0.000000
MntGoldProds           0.000000
NumDealsPurchases      0.000000
NumWebPurchases        0.000000
NumCatalogPurchases    0.000000
NumStorePurchases      0.000000
NumWebVisitsMonth      0.000000
AcceptedCmp3           0.000000
AcceptedCmp4           0.000000
AcceptedCmp5           0.000000
AcceptedCmp1           0.000000
AcceptedCmp2           0.000000
Complain               0.000000
Response               0.000000
dtype: float64

In [7]:
df['Income']=df['Income'].fillna(df['Income'].mean())
df.isnull().sum()

ID                     0
Year_Birth             0
Education              0
Marital_Status         0
Income                 0
Kidhome                0
Teenhome               0
Dt_Customer            0
Recency                0
MntWines               0
MntFruits              0
MntMeatProducts        0
MntFishProducts        0
MntSweetProducts       0
MntGoldProds           0
NumDealsPurchases      0
NumWebPurchases        0
NumCatalogPurchases    0
NumStorePurchases      0
NumWebVisitsMonth      0
AcceptedCmp3           0
AcceptedCmp4           0
AcceptedCmp5           0
AcceptedCmp1           0
AcceptedCmp2           0
Complain               0
Response               0
dtype: int64

### 4. Check if there are any duplicate records in the dataset? If any drop them.

In [8]:
df.duplicated().sum()

0

### 5. Drop the columns which you think redundant for the analysis 

In [9]:
df = df.drop(columns=['ID','Dt_Customer'],axis=1)


### 6. Check the unique categories in the column 'Marital_Status'
- i) Group categories 'Married', 'Together' as 'relationship'
- ii) Group categories 'Divorced', 'Widow', 'Alone', 'YOLO', and 'Absurd' as 'Single'.

In [10]:
df[ 'Marital_Status'].value_counts()



Marital_Status
Married     864
Together    580
Single      480
Divorced    232
Widow        77
Alone         3
Absurd        2
YOLO          2
Name: count, dtype: int64

In [11]:
df[ 'Marital_Status']= df[ 'Marital_Status'].replace(['Married', 'Together'], 'relationship')
df[ 'Marital_Status']= df[ 'Marital_Status'].replace([ 'Divorced', 'Widow', 'Alone', 'YOLO','Absurd' ], 'Single')

In [12]:

df[ 'Marital_Status'].value_counts()

Marital_Status
relationship    1444
Single           796
Name: count, dtype: int64

### 7. Group the columns 'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', and 'MntGoldProds' as 'Total_Expenses'

In [13]:
df['Total_Expenses']=df['MntWines']+df[ 'MntFruits']+df['MntMeatProducts']+df[ 'MntFishProducts']+df['MntSweetProducts']+df['MntGoldProds' ]
df.head(3)


,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,...,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response,Total_Expenses
0,1957,Graduation,Single,58138.0,0,0,58,635,88,546,...,4,7,0,0,0,0,0,0,1,1617
1,1954,Graduation,Single,46344.0,1,1,38,11,1,6,...,2,5,0,0,0,0,0,0,0,27
2,1965,Graduation,relationship,71613.0,0,0,26,426,49,127,...,10,4,0,0,0,0,0,0,0,776


'NumWebP### 8. Group the columns 'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', and 'NumDealsPurchases' as'Num_Total_Purchases'

In [14]:
df[ 'Num_Total_Purchases']=df[ 'NumCatalogPurchases']+df['NumStorePurchases']+df['NumDealsPurchases']+df['NumWebPurchases']
df.head(3)


,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response,Total_Expenses,Num_Total_Purchases
0,1957,Graduation,Single,58138.0,0,0,58,635,88,546,...,7,0,0,0,0,0,0,1,1617,25
1,1954,Graduation,Single,46344.0,1,1,38,11,1,6,...,5,0,0,0,0,0,0,0,27,6
2,1965,Graduation,relationship,71613.0,0,0,26,426,49,127,...,4,0,0,0,0,0,0,0,776,21


### 9. Group the columns 'Kidhome' and 'Teenhome' as 'Kids'

In [15]:
df[ 'Kids']=df[ 'Kidhome' ]+df[  'Teenhome' ]
df.head(3)


,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,...,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response,Total_Expenses,Num_Total_Purchases,Kids
0,1957,Graduation,Single,58138.0,0,0,58,635,88,546,...,0,0,0,0,0,0,1,1617,25,0
1,1954,Graduation,Single,46344.0,1,1,38,11,1,6,...,0,0,0,0,0,0,0,27,6,2
2,1965,Graduation,relationship,71613.0,0,0,26,426,49,127,...,0,0,0,0,0,0,0,776,21,0


### 10. Group columns 'AcceptedCmp1 , 2 , 3 , 4, 5' and 'Response' as 'TotalAcceptedCmp'

In [16]:
df[ 'TotalAcceptedCmp']=df['AcceptedCmp1']+df['AcceptedCmp2']+df['AcceptedCmp3']+df['AcceptedCmp4']+df['AcceptedCmp5']
df.head(3)


,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,...,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response,Total_Expenses,Num_Total_Purchases,Kids,TotalAcceptedCmp
0,1957,Graduation,Single,58138.0,0,0,58,635,88,546,...,0,0,0,0,0,1,1617,25,0,0
1,1954,Graduation,Single,46344.0,1,1,38,11,1,6,...,0,0,0,0,0,0,27,6,2,0
2,1965,Graduation,relationship,71613.0,0,0,26,426,49,127,...,0,0,0,0,0,0,776,21,0,0


### 11. Drop those columns which we have used above for obtaining new features

In [17]:

col_del=[ 'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases',  'NumDealsPurchases','MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts',  'MntGoldProds','Kidhome' ,'Teenhome', 'AcceptedCmp1','AcceptedCmp2','AcceptedCmp3','AcceptedCmp4','AcceptedCmp5' ]
df = df.drop( columns =col_del,axis=1)

df.head(3)


,Year_Birth,Education,Marital_Status,Income,Recency,NumWebVisitsMonth,Complain,Response,Total_Expenses,Num_Total_Purchases,Kids,TotalAcceptedCmp
0,1957,Graduation,Single,58138.0,58,7,0,1,1617,25,0,0
1,1954,Graduation,Single,46344.0,38,5,0,0,27,6,2,0
2,1965,Graduation,relationship,71613.0,26,4,0,0,776,21,0,0


In [18]:

df.shape

(2240, 12)

### 12. Extract 'age' using the column 'Year_Birth' and then drop the column 'Year_birth'

In [19]:
df['Age'] = 2024-df[ 'Year_Birth' ]
df.drop( 'Year_Birth' , axis=1,inplace=True)
df.head(3)

,Education,Marital_Status,Income,Recency,NumWebVisitsMonth,Complain,Response,Total_Expenses,Num_Total_Purchases,Kids,TotalAcceptedCmp,Age
0,Graduation,Single,58138.0,58,7,0,1,1617,25,0,0,67
1,Graduation,Single,46344.0,38,5,0,0,27,6,2,0,70
2,Graduation,relationship,71613.0,26,4,0,0,776,21,0,0,59


In [20]:
#copy of data
dfc=df.copy()


### 13. Encode the categorical variables in the dataset

In [21]:
cate=['Education','Marital_Status']
lbl_encoder=LabelEncoder()
for i in cate:
    df[i]=df[[i]].apply(lbl_encoder.fit_transform)
    
    
df.head(3)
 
        

    

,Education,Marital_Status,Income,Recency,NumWebVisitsMonth,Complain,Response,Total_Expenses,Num_Total_Purchases,Kids,TotalAcceptedCmp,Age
0,2,0,58138.0,58,7,0,1,1617,25,0,0,67
1,2,0,46344.0,38,5,0,0,27,6,2,0,70
2,2,1,71613.0,26,4,0,0,776,21,0,0,59


### 14. Standardize the columns, so that values are in a particular range

In [22]:
df1=df.copy()

Scaled_features = StandardScaler().fit_transform(df1.values)
Scaled_features_df = pd.DataFrame(Standardize_col, index=df1.index ,columns= df.columns)

Scaled_features_df.head()

NameError: name 'Standardize_col' is not defined

### 15. Apply PCA on the above dataset and determine the number of PCA components to be used so that 90-95% of the variance in data is explained by the same.

In [ ]:
##Covariance matrix
cov_matrix = np.cov(scaled_features.T)

##Eigen_values and Eigen vectors
eig_vals ,eig_vectors = np.linalg.eig(cov_matrix)

##scree plot
total = sum(eig_vals)
var_exp = [(i/total)*100 for i in sorted(eig_vals ,reverse=True)]
cum_var_exp = np.cumsum(var_exp)

##Diagram
plt.bar(range(11),var_exp)
plt.bar(range(11),cum_var_exp)
plt.xlabel("principal components")
plt.ylabel("Expalined Varience Ratio")

### 16. Apply K-means clustering and segment the data (Use PCA transformed data for clustering)

In [ ]:
pca=PCA(n_components= 8)
pca_df = pd.DataFrame(pca.fit_transform(scaled_features_df),columns['PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8'])
pca_df.head()

In [ ]:
##Find optimal K using elbow plot

cluster_errors = []
cluster_range =range(2,15)

for num_clusters in cluster_range:
    cluster =KMeans(num_clusters , random_state=100)
    cluster.fit(pca_df)
    cluster_errors.append(clusters.inertia_)
    
cluster_df = pd.DataFrame({'num_clusters':cluster_range,'cluster_errors',:cluster_errors})  
plt.plot(cluster_df['num_clusters'],cluster_df['cluster_errors'],marker='0')

In [ ]:
##Apply KMeans
Kmeans = KMeans(n_clusters=3,randam_state=100)
kmeans.fit(pca_df)

## check cluster labels

In [ ]:
#check the cluster labels
label =pd.DataFrame(Kmeans.label_,columns=['Label'])
Kmeans_df= pca_df.join(label)
Kmeans_df.head()

In [ ]:
Kmeans_df['Label'].values_counts()

In [ ]:
sns.scatterplot(kmeans_df,x='PC1',y='PC2',hue='label',palette='viridis')

### 17. Apply Agglomerative clustering and segment the data (Use Original data for clustering), and perform cluster analysis by doing bivariate analysis between the cluster label and different features and write your observations.

In [ ]:
merege=linkage(scaled_features,method='ward')
dendogram(merge,leaf_rotation=90,)
plot.show()

In [ ]:
from sklearn.metric import silhoutte_score

In [ ]:
for i in range(2,25):
    hier=AgglomeractiveClustering(n_Clusters=i)
    hier=hier.fit(scled_features_df)
    labels=hier.fit_predict(scled_features_df)   
    print(i , silhoutte_score(scled_features_df,labels) )

In [ ]:
hie_cluster = AgglomeractiveClustering(n_Clusters=3, linkage= "ward")
hie_cluster_model =hie_cluster .fit(scled_features_df)

In [ ]:
df_labell =pd.DataFrame(hie_cluster_model.labels_, columns=['labels'])
df_labell.head()

In [ ]:
df_hier = dfc.join(df_labell)
df_hier.head()

### Visualization and Interpretation of results

-----
## Happy Learning
-----